<a href="https://colab.research.google.com/github/rayenTech/MQTT_QUIC_Security/blob/main/MQTT_QUIC_Security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Installer les bibliothèques pour MQTT
!pip install paho-mqtt

# Installer les bibliothèques pour QUIC
!pip install aioquic

# Installer les bibliothèques pour la visualisation et le traitement des données
!pip install matplotlib numpy

# Installer les bibliothèques pour requêtes HTTP (si besoin)
!pip install requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 13.1 MB/s eta 0:00:00


In [ ]:
import paho.mqtt.client as mqtt
import time
import json

# Remplacer par votre API Key MQTT (masquée pour la sécurité)
MQTT_API_KEY = "REDACTED"
CHANNEL_ID = "REDACTED"

# Configurer le client MQTT
MQTT_BROKER = "mqtt.thingspeak.com"
MQTT_PORT = 1883  # Port par défaut MQTT
TOPIC = f"channels/{CHANNEL_ID}/publish/{MQTT_API_KEY}"

# Données simulées
intrusion_attempts = [0, 1, 2, 1, 3, 4, 2, 0, 5, 1]        # Intrusion attempts
antivirus_status = [1, 0, 1, 1, 0, 1, 1, 0, 1, 1]          # Antivirus status (1 = active, 0 = inactive)
security_alerts = [5, 3, 4, 2, 6, 7, 3, 4, 5, 2]           # Security alerts
cpu_usage = [20.5, 30.0, 25.5, 40.0, 35.0, 50.0, 45.5, 30.0, 60.0, 55.0] # CPU usage in percentage
login_attempts = [10, 12, 8, 15, 20, 18, 25, 30, 22, 17]   # Login attempts
failed_logins = [2, 1, 3, 0, 4, 5, 2, 3, 1, 0]             # Failed logins
firewall_blocks = [1, 0, 2, 3, 4, 5, 6, 7, 2, 3]           # Firewall blocks
defender_usage = [75.0, 80.5, 70.0, 85.0]                  # Microsoft Defender usage in %

# Initialiser le client MQTT
client = mqtt.Client()

def send_data(client, field_values):
    payload = json.dumps(field_values)
    client.publish(TOPIC, payload)

# Se connecter au broker MQTT
client.connect(MQTT_BROKER, MQTT_PORT)

# Envoyer des données dans une boucle
for i in range(len(intrusion_attempts)):
    data = {
        "field1": intrusion_attempts[i],
        "field4": cpu_usage[i]
    }
    send_data(client, data)
    print(f"Data sent: {data}")
    time.sleep(15)  # Respecter les limitations d'envoi


In [ ]:
import paho.mqtt.client as mqtt
import json

# Remplacer par votre API Key MQTT et CHANNEL ID
MQTT_API_KEY = "REDACTED"
CHANNEL_ID = "REDACTED"

# Configurer le client MQTT
MQTT_BROKER = "mqtt.thingspeak.com"
MQTT_PORT = 1883
SUBSCRIBE_TOPIC = f"channels/{CHANNEL_ID}/subscribe/fields/field1/{MQTT_API_KEY}"

# Callback lorsque des messages arrivent
def on_message(client, userdata, message):
    payload = json.loads(message.payload.decode())
    print("Data received:")
    print(payload)

# Initialiser le client MQTT
client = mqtt.Client()
client.on_message = on_message

# Connexion au broker MQTT
client.connect(MQTT_BROKER, MQTT_PORT)

# Souscrire au sujet
client.subscribe(SUBSCRIBE_TOPIC)

# Récupérer les messages
print("Waiting for data...")
client.loop_forever()


In [ ]:
import asyncio
from aioquic.asyncio import connect
from aioquic.asyncio.protocol import QuicConnectionProtocol
import matplotlib.pyplot as plt

# URL et port pour QUIC
QUIC_HOST = "REDACTED"
QUIC_PORT = 443
QUIC_PATH = "/channels/REDACTED/feeds.json"

class QuicClientProtocol(QuicConnectionProtocol):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = None

    def quic_event_received(self, event):
        if hasattr(event, 'data') and event.data:
            self.data = event.data.decode()

async def fetch_data():
    async with connect(QUIC_HOST, QUIC_PORT, protocol_factory=QuicClientProtocol) as protocol:
        stream_id = protocol._quic.get_next_available_stream_id()
        request = f"GET {QUIC_PATH} HTTP/1.1\r\nHost: {QUIC_HOST}\r\n\r\n"
        protocol._quic.send_stream_data(stream_id, request.encode(), end_stream=True)
        await protocol.wait_closed()
        return protocol.data

# Affichage des données
async def main():
    data = await fetch_data()
    feeds = data["feeds"]

    intrusion_attempts = [int(feed["field1"]) for feed in feeds if feed["field1"]]
    cpu_usage = [float(feed["field4"]) for feed in feeds if feed["field4"]]

    plt.plot(intrusion_attempts, label="Intrusion Attempts")
    plt.plot(cpu_usage, label="CPU Usage")
    plt.legend()
    plt.show()

asyncio.run(main())
